In [3]:
import pandas as pd

df = pd.read_csv('../../DSI-CHI-1/lessons/week-06/3.1-random-forests-boosting/assets/datasets/car.csv')
print df.shape
df.head()

(1728, 7)


,buying,maint,doors,persons,lug_boot,safety,acceptability
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [10]:
for col in df:
    print df[col].value_counts()
    print df[col].describe()
    print ''

med      432
high     432
low      432
vhigh    432
Name: buying, dtype: int64
count     1728
unique       4
top        med
freq       432
Name: buying, dtype: object

med      432
high     432
low      432
vhigh    432
Name: maint, dtype: int64
count     1728
unique       4
top        med
freq       432
Name: maint, dtype: object

3        432
5more    432
4        432
2        432
Name: doors, dtype: int64
count     1728
unique       4
top          3
freq       432
Name: doors, dtype: object

more    576
4       576
2       576
Name: persons, dtype: int64
count     1728
unique       3
top       more
freq       576
Name: persons, dtype: object

med      576
small    576
big      576
Name: lug_boot, dtype: int64
count     1728
unique       3
top        med
freq       576
Name: lug_boot, dtype: object

med     576
high    576
low     576
Name: safety, dtype: int64
count     1728
unique       3
top        med
freq       576
Name: safety, dtype: object

unacc    1210
acc       384
good   

In [6]:
df.isnull().any()

buying           False
maint            False
doors            False
persons          False
lug_boot         False
safety           False
acceptability    False
dtype: bool

In [11]:
from sklearn.preprocessing import LabelEncoder
y = LabelEncoder().fit_transform(df['acceptability'])
X = pd.get_dummies(df.drop('acceptability', axis=1))

In [12]:
X.head()

,buying_high,buying_low,buying_med,buying_vhigh,maint_high,maint_low,maint_med,maint_vhigh,doors_2,doors_3,...,doors_5more,persons_2,persons_4,persons_more,lug_boot_big,lug_boot_med,lug_boot_small,safety_high,safety_low,safety_med
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [25]:
y

array([2, 2, 2, ..., 2, 1, 3])

In [26]:
from sklearn.cross_validation import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier

cv = StratifiedKFold(y, n_folds=3, shuffle=True, random_state=41)

In [27]:
cv

sklearn.cross_validation.StratifiedKFold(labels=[2 2 2 ..., 2 1 3], n_folds=3, shuffle=True, random_state=41)

In [28]:
dt = DecisionTreeClassifier(class_weight='balanced')
s = cross_val_score(dt, X, y, cv=cv, n_jobs=-1)

In [29]:
s

array([ 0.95667244,  0.97916667,  0.95652174])

In [30]:
print "{} Score:\t{:0.3} ± {:0.3}".format("Decision Tree", s.mean().round(3), s.std().round(3))

Decision Tree Score:	0.964 ± 0.011


In [31]:
bdt = BaggingClassifier(DecisionTreeClassifier())
rf = RandomForestClassifier(class_weight='balanced', n_jobs=-1)
et = ExtraTreesClassifier(class_weight='balanced', n_jobs=-1)

In [32]:
def score(model, name):
    s = cross_val_score(model, X, y, cv=cv, n_jobs=-1)
    print "{} Score:\t{:0.3} ± {:0.3}".format(name, s.mean().round(3), s.std().round(3))

In [33]:
score(dt, "Decision Tree")
score(bdt, "Bagging DT")
score(rf, "Random Forest")
score(et, "Extra Trees")

Decision Tree Score:	0.964 ± 0.011
Bagging DT Score:	0.969 ± 0.006
Random Forest Score:	0.95 ± 0.004
Extra Trees Score:	0.961 ± 0.009


In [34]:
et.fit(X, y)

ExtraTreesClassifier(bootstrap=False, class_weight='balanced',
           criterion='gini', max_depth=None, max_features='auto',
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [35]:
predictions = et.predict(X)

In [36]:
from sklearn.metrics import classification_report, confusion_matrix

print confusion_matrix(y, predictions)

[[ 384    0    0    0]
 [   0   69    0    0]
 [   0    0 1210    0]
 [   0    0    0   65]]


In [37]:
print classification_report(y, predictions)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       384
          1       1.00      1.00      1.00        69
          2       1.00      1.00      1.00      1210
          3       1.00      1.00      1.00        65

avg / total       1.00      1.00      1.00      1728



In [38]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
ab = AdaBoostClassifier()
gb = GradientBoostingClassifier()
score(ab, "AdaBoost")
score(gb, "Gradient Boosting Classifier")

AdaBoost Score:	0.811 ± 0.002
Gradient Boosting Classifier Score:	0.982 ± 0.006
